In [ ]:
import os
import shutil

# Define the path to your dataset and create directories for blood and normal images
dataset_path = "dataset"  # Update with your dataset path
blood_dir = os.path.join(dataset_path, "blood")
normal_dir = os.path.join(dataset_path, "normal")

# Create directories for blood and normal images if they don't exist
os.makedirs(blood_dir, exist_ok=True)
os.makedirs(normal_dir, exist_ok=True)

# Iterate through each image in the dataset
for filename in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, filename)
    # Ensure we're working with files and not directories
    if os.path.isfile(file_path):
        # Classify based on the filename pattern
        if filename.lower().startswith("blood"):
            shutil.move(file_path, os.path.join(blood_dir, filename))
        elif filename[0].isupper() or filename[0].isdigit():
            shutil.move(file_path, os.path.join(normal_dir, filename))

print("Images classified into 'blood' and 'normal' folders successfully.")


Images classified into 'blood' and 'normal' folders successfully.


In [4]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Step 1: Organize the Dataset into Blood and Normal Folders
dataset_path = "dataset"  # Update this path to your dataset
blood_dir = os.path.join(dataset_path, "blood")
normal_dir = os.path.join(dataset_path, "normal")

# Create directories if they don't exist
os.makedirs(blood_dir, exist_ok=True)
os.makedirs(normal_dir, exist_ok=True)

# Separate images based on filename
for filename in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):
        if filename.lower().startswith("blood"):
            shutil.move(file_path, os.path.join(blood_dir, filename))
        elif filename[0].isupper() or filename[0].isdigit():
            shutil.move(file_path, os.path.join(normal_dir, filename))

print("Images have been organized into 'blood' and 'normal' folders.")

# Step 2: Set up Data Generators
batch_size = 32
img_height = 128
img_width = 128

# Data augmentation for training and validation
data_gen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

train_data = data_gen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = data_gen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Step 3: Define the CNN Model for Blood Detection
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Step 4: Train the Model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Save the trained model
model.save("blood_detection_model.h5")
print("Model saved as 'blood_detection_model.h5'")

# Step 5: Evaluate the Model
val_steps = val_data.samples // batch_size
val_data.reset()
predictions = model.predict(val_data, steps=val_steps, verbose=1)
pred_labels = (predictions > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(val_data.classes, pred_labels, target_names=["Normal", "Blood"]))

print("\nConfusion Matrix:")
print(confusion_matrix(val_data.classes, pred_labels))


Images have been organized into 'blood' and 'normal' folders.
Found 1105 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


/home/diva/Documents/Github/road safety/test/test/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1730920719.808458  324403 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/diva/Documents/Github/road safety/test/test/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` ca

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 491ms/step - accuracy: 0.6336 - loss: 0.7182 - val_accuracy: 0.9345 - val_loss: 0.2141
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 17s 467ms/step - accuracy: 0.9238 - loss: 0.2041 - val_accuracy: 0.9418 - val_loss: 0.1615
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 17s 470ms/step - accuracy: 0.9433 - loss: 0.1550 - val_accuracy: 0.8873 - val_loss: 0.2940
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 456ms/step - accuracy: 0.9514 - loss: 0.1170 - val_accuracy: 0.9382 - val_loss: 0.1912
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 462ms/step - accuracy: 0.9540 - loss: 0.1230 - val_accuracy: 0.9491 - val_loss: 0.2175
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 457ms/step - accuracy: 0.9675 - loss: 0.0841 - val_accuracy: 0.9418 - val_loss: 0.2027
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 462ms/step - accuracy: 0.9672 - loss: 0.0820 - val_accuracy: 0.9309 - val_loss: 0.2237
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 452ms/step - accuracy: 0.9790 - loss: 0.0498 - val_accu

Model saved as 'blood_detection_model.h5'
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 323ms/step

Classification Report:


ValueError: Found input variables with inconsistent numbers of samples: [275, 256]

In [6]:

import tensorflow as tf
import numpy as np
batch_size = 32
img_height = 128
img_width = 128


In [7]:
# Load the model
model = tf.keras.models.load_model("blood_detection_model.h5")

# Predict function for new images
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    prediction = model.predict(img_array)[0][0]
    label = "Blood" if prediction > 0.5 else "Normal"
    return label, prediction

# Example prediction
image_path = "1.jpg"  # Update with the path to your test image
label, score = predict_image(image_path)
print(f"Prediction: {label} (Confidence: {score:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Prediction: Normal (Confidence: 0.41)


In [8]:
# Load the model
model = tf.keras.models.load_model("blood_detection_model.h5")

# Predict function for new images
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    prediction = model.predict(img_array)[0][0]
    label = "Blood" if prediction > 0.5 else "Normal"
    return label, prediction

# Example prediction
image_path = "2.jpg"  # Update with the path to your test image
label, score = predict_image(image_path)
print(f"Prediction: {label} (Confidence: {score:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Prediction: Blood (Confidence: 1.00)


In [11]:
import cv2
import tensorflow as tf
import numpy as np

# Parameters
img_height = 128
img_width = 128  

# Load the pre-trained model
model = tf.keras.models.load_model("blood_detection_model.h5")

# Function to predict blood presence in a single image array
def predict_image(img_array):
    # Resize and normalize the image to fit the model input
    img_array = cv2.resize(img_array, (img_width, img_height)) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Predict using the model
    prediction = model.predict(img_array)[0][0]
    label = "Blood" if prediction > 0.5 else "Normal"
    return label

# Process the video frame by frame and determine the final result
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    blood_detected = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Predict on the current frame
        label = predict_image(frame)
        
        # Check if blood is detected in this frame
        if label == "Blood":
            blood_detected = True

    cap.release()
    
    # Final output based on detection
    if blood_detected:
        print("Overall result: Blood was detected in the video")
    else:
        print("Overall result: No blood detected in any frame")

# Example usage
video_path = "video.mp4"  # Update with the path to your video
process_video(video_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━

In [15]:
import cv2
import numpy as np

# Placeholder for a blood detection function based on RGB values
def detect_blood_in_frame(frame):
    # Convert frame to the RGB color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define blood-like color range in HSV (adjust these values as needed)
    lower_blood = np.array([0, 70, 50])    # Dark red hues
    upper_blood = np.array([10, 255, 255])  # Bright red hues
    
    # Create a binary mask for areas that match the blood-like color
    blood_mask = cv2.inRange(hsv, lower_blood, upper_blood)
    
    # Refine detection based on mask size to reduce noise
    blood_detected = np.sum(blood_mask) > 5000  # Adjust threshold for detection
    
    return blood_detected, blood_mask

# Process the video frame by frame
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    blood_detected_in_video = False

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply blood detection on the current frame
        blood_detected, blood_mask = detect_blood_in_frame(frame)
        
        if blood_detected:
            blood_detected_in_video = True
            # Optional: Highlight detected areas on the frame
            frame[blood_mask > 0] = [0, 0, 255]  # Red overlay for blood areas
        
        # Display the frame (optional for real-time viewing)
        cv2.imshow("Blood Detection", frame)
        
        # Exit the video display window on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    # Final output based on detection
    if blood_detected_in_video:
        print("Overall result: Blood was detected in the video")
    else:
        print("Overall result: No blood detected in any frame")

# Example usage
video_path = "video.mp4"  # Update with the path to your video
process_video(video_path)


Overall result: Blood was detected in the video
